<a href="https://colab.research.google.com/github/anthonyhu25/Bayesian-Queuing-/blob/main/Bayesian_Queueing_Inference_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numpy import random
from numpy import linalg
import math
import scipy
import scipy.stats
import matplotlib.pyplot as plt
from scipy.stats import rv_continuous, rv_discrete
from scipy.stats._distn_infrastructure import rv_frozen
from scipy.special import logsumexp
import warnings
import sys
import statistics
import pandas as pd
from IPython.display import display, Math, HTML
import warnings
warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)

#1. Introduction


For the $M/G/1$ queueing model, customers arrive with independent interarrival times $W_{i} \sim Exp(\theta_{3}) = \theta_{3}e^{-\theta_{3}w_{i}}$

Customers have a service time $U_{i} \sim Uniform(\theta_{1}, \theta_{2})$ distribution. We only observe interdeparture times $Y_{i}$. Purpose of model is to infer the parameters $\theta = (\theta_{1}, \theta_{2}, \theta_{3})$ from the observed interdeparture times $y = (y_{1},...,y_{n})$.

We can represent the interdeparture times $Y_{i}$ as:

$\large Y_{i} = U_{i} + max(0, \sum_{j=1}^{i}W_{j} - \sum_{j=1}^{i-1}Y_{j}) = U_{i} + max(0, V_{i}-X_{i-1})$

where $X_{i} = \sum_{j=1}^{i}$ is the departure time of $i$-th customer, $V_{i}= \sum_{j=1}^{i}W_{j}$ is the arrival time of $i$-th customer, and $W_{j}$ difference in time between arrival times of $j$-th customer and $(j-1)$-th customer.

In the paper, they view the arrival times $V_{i}$ as latent(unobserved) variables, and summarized the conditional distributions with regards to previous arrival times and departure times. But, we need to sample $V_{i}$ conditional on the observed interdepature times $y$, and the previous $V_{i}$ samples (we denote this as $V_{-i}$)

We sample the queuing model $V_{i}$ as follows:

$V_{1} \sim Exp(\theta_{3})$

$V_{i}|V_{i-1} \sim V_{i-1} + Exp(\theta_{3})$

$Y_{i}|X_{i}, V_{i} \sim Uniform(\theta_{1} + max(0, V_{i} - X_{i-1}), \theta_{2} + max(0, V_{i} - X_{i-1})$ for $ i = 1,...,n$

## 2.1 Gibbs update for arrival time

To sample the $V_{i}$, we will use standard Gibbs sampling updates, which will need the conditional density of $V_{i}$ given other $V_{i}$-s and $\theta$.

There are 3 cases presented in the paper: when $i = 1$, $2 \le i \le n-1$, and $i = n$

### Case 1: $i = 1$

$\large P(v_{1}|v_{-1}, y, θ) ∝ P(v_{1})P(y_{1}|v_{1},θ)P(v_{2}|v_{1},θ)$

$\large ∝ [\theta_{3}exp(-\theta_{3}v_{1})I(v_{1} \ge 0)]\frac{I(y_{1} ∈ [\theta_{1} + v_{1}, \theta_{2} + v_{1}])}{\theta_{2}-\theta_{1}}[\theta_{3}exp(-\theta_{3}(v_{2}-v_{1}))I(v_{1} \le v_{2})]$

$\large \propto I(v_{1} \in [max(0, x_{1} - \theta_{1}), min(v_{2}, x_{1} - \theta_{1})] \sim Uniform(max(0, x_{1} - \theta_{1}), min(v_{2}, x_{1} - \theta_{1}))$

Note that the last line comes from rewriting the inequalities $\theta_{1} + v_{1} \le y_{1} \le \theta_{2} + v_{1}$ in terms of $v_{1}$, and also taking into consideration of the bounds $I(v_{1} \ge 0)$ and $I(v_{1} \le v_{2})$

### Case 2: $2 \le i \le n - 1$

Note that $V_{i} - X_{i-1}$ is the time between the arrival of the ${i}$-th customer, and the departure of the $(i-1)$-th customer. So we are sampling interdeparture time $Y_{i}$, conditional on the arrival $V_{i}$ and $X_{i-1}$, and the service time in one step for the uniform service distribution.

Intuitively speaking, we know the arrival times $V_{i-1}$ and $V_{i+1}$, as well as all of the interdeparture times $Y_{i}$ and parameters $\theta$. So we expres probability of $V_{i}$ in terms of the service distribution, shifted based on the location of $V_{i-1}$ and $V_{i+1}$.

$\large P(v_{i}|v_{-i}, y, θ) ∝ P(v_{i}|v_{i-1}, θ)P(y_{i}|x_{i-1}, v_{i}, \theta)P(v_{i+1}|v_{i}, θ)$

$\large \propto \theta_{3}exp(-\theta_{3}(v_{i}-v_{i-1}))I(v_{i-1} \le v_{i})[\frac{y_{i} \in [\theta_{1} + max(0, v_{i}-x_{i-1}), \theta_{2} + max(0, v_{i} - x_{i-1})]}{\theta_{2} - \theta_{1}}] \theta_{3}exp(-\theta_{3}(v_{i+1}-v_{i}))I(v_{i} \le v_{i+1})$

$\large \propto I(v_{i} \in [v_{i-1}, v_{i+1}])I(y_{i} \in [\theta_{1} + max(0, v_{i}-x_{i-1}), \theta_{2} + max(0, v_{i} - x_{i-1})])$

For this expression, we consider two cases: $Y_{i} > \theta_{2}$, and $Y_{i} \le \theta_{2}$, since we seek to establish the behavior of $Y_{i}$ conditional on the service distribution $U_{i}$, and as mentioned before $Y_{i} = U_{i} + max(0, V_{i} - X_{i-1})$

Since $U_{i} \sim Uniform(\theta_{1}, \theta_{2})$, $U_{i} \le \theta_{2}$ for all i. Hence, we examine $Y_{i}$ based on $\theta_{2}$.

If $Y_{i} > \theta_{2}$, this implies that $V_{i} > X_{i-1}$. So, $Y_{i} \sim Uniform(X_{i} - \theta_{1}, max(V_{i+1}, X_{i}-\theta_{2}))$ by rewriting the uniform distribution, which was in terms of $Y_{i}$, into terms of $V_{i}$.

If $Y_{i} \le \theta_{2}$, we split the indicator into regions based on whether $v_{i}$ is lesss than or equal to $x_{i}$. The final uniform distribution simplifies to $I(v_{i} \in [v_{i-1}, min(v_{i+1}, x_{i} - \theta_{1})])$

So for both cases, the arrival time of $i$-th customer $V_{i} \sim Uniform(v_{i-1}, min(v_{i+1}, x_{i} - \theta_{1}))$.

### Case 3: $i = n$

$\large P(v_{n}|v_{-n}, y, \theta) \propto P(v_{n}|v_{n-1})P(y_{n}|v_{n})$

$\propto exp(-\theta_{3}(v_{n}-v_{n-1}))I(v_{n-1} \le v_{n})I(y_{n} \in [\theta_{1} + max(0, v_{n}-x_{n-1}), \theta_{2} + max(0, v_{n} - x_{n-1})])$

If $y_{n} > \theta_{2}$, then $V_{n}$ will have a truncated Exponential distribution on $[x_{n} - \theta_{2}, x_{n} - \theta_{1}]$. Else, $V_{n}$ will have a truncated Exponential distribution on $[v_{n-1}, x_{n}-\theta_{1}]$. In either case, $V_{n}$ has rate $\theta_{3}$.

Note that we sample $V_{i}$ where $i = n$ by using the inverse CDF trick on the truncated exponential. Let $[L,U]$ be the lower and upper bound of the interval, and CDF be:

$\large F_{V_{n}}(v_{n}) = \frac{e^{-\theta_{3}L} - e^{-\theta_{3}v_{n}}}{e^{-\theta_{3}L} - e^{-\theta_{3}U}}$

Its inverse cdf:

$\large F_{V_{n}}^{-1}(y) = \frac{-1}{\theta_{3}}log((1-y)e^{-\theta_{3}L} + ye^{-\theta_{3}U})$

To sample from the truncated exponential, take $R\sim Uniform(0,1)$ and find value $F_{V_{n}}^{-1}(R)$.

In [78]:
def truncated_exponential_sampler(U, rate, lower, upper):
  return (- 1 /rate) *  np.log((1 - U) * np.exp(-rate * lower) + U * np.exp(- rate * upper))

In [91]:
def gibbs_sampler_queue(y, n, v, theta_dict):
  # length of y vector and v vector must be equal to n
  if len(y) != n:
    raise AttributeError("Length of y-vector is not equal to n")
  if len(v) != n:
    raise AttributeError("Length of v-vector is not equal to n")
  samples = []
  for i in range(n):
    x_i = np.cumsum(y)[i]
    y_i = y[i]
    if i == 0:
      lower = max(0, x_i - theta_dict['theta_2'])
      upper = min(v[0], x_i - theta_dict['theta_1'])
      posterior_sample = np.random.uniform(low = lower, high = upper, size = 1)
    elif i == n-1:
      if y_i > theta_dict['theta_2']:
        lower = x_i - theta_dict['theta_2']
      else:
        lower = v[i-1]
      upper = x_i - theta_dict['theta_1']
      uniform = np.random.uniform(low = 0, high = 1, size = 1)
      posterior_sample = truncated_exponential_sampler(uniform, theta_dict['theta_3'], lower, upper)
    else:
      if y_i <= theta_dict['theta_2']:
        lower = v[i-1]
      else:
        lower = x_i - theta_dict['theta_2']
      upper = min(v[i+1], x_i - theta_dict['theta_1'])
      posterior_sample = np.random.uniform(low = lower, high = upper, size = 1)
    samples.append(posterior_sample)
  return [x.item() for x in samples]

In [9]:
def simulate_v(theta_dict, N, y):
  v_sim = []
  y_sim = []
  for i in range(N):
    X_prev = sum(y[:(i)])
    if len(v_sim) == 0:
      V_i = scipy.stats.expon.rvs(scale = theta_dict['theta_3'], size = 1)
      v_sim.append(float(V_i))
    else:
      V_prev = v_sim[i-1]
      V_i = scipy.stats.expon.rvs(scale = theta_dict['theta_3'], size = 1) + V_prev
      v_sim.append(float(V_i))
    lower = theta_dict['theta_1'] + max(0, V_i - X_prev)
    upper = theta_dict['theta_2'] + max(0, V_i - X_prev)
    Y_i = scipy.stats.uniform.rvs(loc = lower, scale = upper - lower, size = 1)
    y_sim.append(float(Y_i))
  return v_sim, y_sim

In [4]:
def simulate_v_np(theta_dict, N, y):
  v_sim = []
  y_sim = []
  for i in range(N):
    X_prev = sum(y[:(i)])
    if len(v_sim) == 0:
      V_i = np.random.exponential(scale = theta_dict['theta_3'], size = 1)
      v_sim.append(float(V_i))
    else:
      V_prev = v_sim[i-1]
      V_i = np.random.exponential(scale = theta_dict['theta_3'], size = 1) + V_prev
      v_sim.append(float(V_i))
    lower = theta_dict['theta_1'] + max(0, V_i - X_prev)
    upper = theta_dict['theta_2'] + max(0, V_i - X_prev)
    Y_i = scipy.stats.uniform.rvs(loc = lower, scale = upper - lower, size = 1)
    y_sim.append(float(Y_i))
  return v_sim, y_sim

In [5]:
theta_intermediate = {
    'theta_1': 4,
    'theta_2' :7,
    'theta_3': 0.15
}

In [6]:
Frequent = [
11.57, 13.44, 13.24, 9.30, 8.95, 11.99, 15.68, 10.72, 12.68, 9.79,
14.01, 10.04, 12.05, 13.59, 15.13, 15.67, 12.38, 9.11, 9.19, 10.06,
14.73, 10.03, 14.51, 9.95, 15.43, 10.80, 9.57, 10.01, 12.93, 11.79,
10.81, 14.65, 12.68, 12.40, 15.34, 10.29, 14.06, 14.03, 11.04, 12.54,
8.61, 8.43, 12.25, 14.23, 15.47, 9.04, 12.55, 11.76, 8.10, 10.70
]

Intermediate = [
6.19, 6.04, 9.52, 4.49, 4.36, 9.86, 9.91, 5.02, 5.76, 4.67,
6.25, 4.77, 5.52, 6.10, 6.67, 6.88, 5.64, 4.42, 4.45, 4.77,
6.52, 4.76, 6.44, 4.73, 6.79, 5.05, 4.59, 4.75, 5.85, 5.42,
5.05, 6.49, 5.76, 8.67, 16.65, 4.86, 6.27, 6.26, 5.14, 10.60,
4.23, 6.15, 5.59, 6.34, 6.80, 4.39, 5.71, 5.41, 4.04, 5.01
]

Rare = [
21.77, 10.30, 206.34, 8.57, 45.79, 233.13, 128.30, 59.73, 4.59, 3.21,
185.29, 2.49, 4.63, 72.48, 22.47, 195.34, 85.92, 8.39, 23.30, 4.24,
42.78, 332.64, 16.91, 6.26, 39.44, 27.16, 29.53, 93.65, 42.60, 176.36,
34.69, 345.20, 128.16, 307.50, 233.54, 18.79, 36.88, 114.85, 4.73, 337.02,
81.89, 96.33, 27.20, 23.16, 167.89, 70.58, 81.28, 43.55, 33.88, 28.47
]

In [99]:
# Testing intermediate queue
v_sim, y_sim = simulate_v(theta_intermediate, len(Intermediate), Intermediate)
x_sim = np.cumsum(y_sim)
print(x_sim)
print(np.cumsum(Intermediate))

[  4.92877529   9.22746924  13.23860781  20.23318906  24.3961362
  28.6589288   33.34375416  38.43957365  45.06778767  49.70247663
  53.7153894   57.89078115  63.95497725  68.48311741  72.99117435
  78.93316817  83.61733539  89.4771629   94.27945488 101.16694521
 106.81084383 112.20813288 118.47048206 125.03059399 130.9629192
 135.0291697  140.67094677 147.27181562 154.17316625 159.16533588
 164.11534557 170.92999653 176.19096469 182.69245174 187.84311233
 192.16790663 197.20149238 203.17677474 209.39128145 214.29177584
 219.29935532 225.71207554 229.95194259 234.70028208 239.87733726
 246.6682166  252.61501564 258.14896568 264.66140189 271.23377501]
[  6.19  12.23  21.75  26.24  30.6   40.46  50.37  55.39  61.15  65.82
  72.07  76.84  82.36  88.46  95.13 102.01 107.65 112.07 116.52 121.29
 127.81 132.57 139.01 143.74 150.53 155.58 160.17 164.92 170.77 176.19
 181.24 187.73 193.49 202.16 218.81 223.67 229.94 236.2  241.34 251.94
 256.17 262.32 267.91 274.25 281.05 285.44 291.15 296.56 

In [103]:
gibbs_intermediate = gibbs_sampler_queue(Intermediate, len(Intermediate), v_sim, theta_intermediate)
print(np.cumsum(gibbs_intermediate))

[  0.00192707   0.07931138   1.73645462   2.28462986   2.89774664
  12.42526847  45.38078609  46.4173694   47.66122356  49.1173476
  51.01824951  52.6855595   55.01169245  57.6284835   60.28069222
  63.11357535  66.02432372  69.09550597  72.29873371  75.47395083
  78.76466379  82.1396331   85.51565106  89.07961921  92.6633453
  96.31651952 100.05106124 103.80855864 107.66231698 111.76680394
 115.96840512 120.31762626 124.84500118 223.53090229 265.94560735
 270.70811116 275.61195632 280.84425202 285.95649776 405.05687405
 410.80078527 416.98535145 423.34286087 429.7667702  436.29964839
 443.60615263 450.67154581 458.18290144 465.77908651 473.41016575]


In [60]:
# Testing frequent queue
theta_frequent = {
    'theta_1': 8,
    'theta_2': 16,
    'theta_3': 0.15
}
v_sim2, y_sim2 = simulate_v(theta_frequent, len(Frequent), Frequent)
print(v_sim2)
print(y_sim2)
x_sim2 = np.cumsum(y_sim2)
print(x_sim2)
print(np.cumsum(Frequent))

[0.007319861679733879, 0.25105473824929697, 0.5220043819257669, 0.6292514138433443, 0.6879474536378776, 1.2313014679875935, 1.2442227601293019, 1.3738382955374873, 1.4793000185786347, 1.5259245381450737, 1.545931009344652, 1.6323933556644439, 1.7929201118365872, 1.8479427995970106, 2.005730541617945, 2.3931519497716036, 2.4109024415303164, 2.450616613357885, 2.4769348153218496, 2.5403136951571446, 2.551322668018276, 2.5559197145709955, 2.6190163677194715, 2.6558685504962267, 2.6999781126336266, 3.0083113299507045, 3.013386410718605, 3.0665740019495646, 3.218759942117885, 3.375869602358648, 3.5264165133517595, 3.5601717196202722, 3.697335281620958, 3.728049049688901, 3.820224746630258, 3.9411342131186813, 3.9829271623245486, 4.004949578202043, 4.089368389963046, 4.110905116478038, 4.191616366796431, 4.34850053311896, 4.4140021109637, 4.56983873500135, 4.737372492945928, 4.821163936895414, 5.076482066279807, 5.326230722287326, 5.358644922619393, 5.567248448291089]
[13.791415116628134, 9.

# 2.2 Simple Metropolis updates for parameters

Now, after $V_{i}$ and the data are available, we would like to sample from the conditional posterior distribution of $\theta \sim \pi(\theta|v,y) \propto \pi(v, \theta|y) \propto \pi(\theta)P(v_{1}|\theta) \prod_{i=2}^{n}P(v_{i}|v_{i-1}, \theta) \prod_{i=1}^{n}P(y_{i}|v_{i}, x_{i-1}, \theta)$.

We reparameterize $\theta$ to $\eta$, where $\eta = (\eta_{1}, \eta_{2}, \eta_{3}) = (\theta_{1}, \theta_{2}-\theta_{1}, log(\theta_{3}))$.

Then, we choose a symmetric proposal density $q(.|\eta)$ : $q(x|x^{̃}) ≡ q(x^{̃}|x)$. Then, we generate proposal $\eta^{*} \sim q(\eta^{*}|\eta)$, and compute acceptance probability

$\large a = min(1, \frac{\pi(\eta^{*}|v,y)}{\pi(\eta|v,y)})$.

We use a normal proposla with independent coordinates centered around current value of $\eta$. If proposed value for $\eta_{1}$ or $\eta_{2}$ is outside of its range, we immediately reject the proposal.

The logarithm of posterior simplifies to
$\large log \pi(v, \theta|y) = log(\pi(\theta)) + nlog(\theta_{3}) - \theta_{3}v_{n} - nlog(\theta_{2} - \theta_{1})$

With these constraints (due to the bounds on original parameters $\theta$):

$\theta_{2} > \theta_{1}$

$\theta_{1} \le y_{1}-v_{1}$

$\theta_{2} \ge y_{1}-v_{1}$

$\theta_{1} \le min(y_{i} - max(0, v_{i}-x_{i-1}))$ for $i \ge 2$

$\theta_{2} \ge max(y_{i} - max(0, v_{i} - x_{i-1}))$ for $i \ge 2$

Note that the paper places a $Uniform(0, 10)$ prior on $\theta_{1}$ and on $\theta_{2} - \theta_{1}$, and a $Uniform(0, 1/3)$ prior on $\theta_{3}$. So, the joint prior $\pi(\theta)$ for $\theta$ is:

$\large \pi(\theta) \propto I(\theta_{1} \in [0,10]) I(\theta_{2}-\theta_{1} \in [0,10])I(\theta_{3} \in [0,1/3])$

With the $\eta$ parameterization, priors on $\eta_{1}$ and $\eta_{2}$ remain the same, and $\eta_{3}$ is now proportional to $exp(\eta_{3})$ on $(-∞, log(1/3))$ and $0$ otherwise.

In [106]:
# theta should be a dictionary
def log_posterior(theta, v, y):
  n = len(v)
  indicator_constraint = all([0 <= theta['theta_1'] <= 10,
                              0 <= theta['theta_2'] - theta['theta_1'] <= 10,
                              0 <= theta['theta_3'] <= 1/3])
  constraint_1 = theta['theta_2'] > theta['theta_1']
  constraint_2 = theta['theta_1'] <= y[0] - v[0]
  constraint_3 = theta['theta_2'] >= y[0] - v[0]
  # pre-define z_i as y_i - max(0, v_i - x_i-1)
  x = np.cumsum(y)
  z = [float(y[i] - max(0, v[i] - x[i-1])) for i in range(1, n)]
  constraint_4 = theta['theta_1'] <= min(z)
  constraint_5 = theta['theta_2'] >= max(z)
  print(z)
  print([constraint_4, constraint_5])
  if all([indicator_constraint, constraint_1, constraint_2, constraint_3, constraint_4, constraint_5]) == True:
    log_posterior = n * np.log(theta['theta_3']) -\
     theta['theta_3'] * v[-1] -\
      n * np.log(theta['theta_2'] - theta['theta_1'])
  else:
    log_posterior = -np.inf
  return log_posterior

In [104]:
def metropolis_simple(eta_sd, theta_dict, v, y, n_iterations, n_burn_in):
  MC_chain = []
  for i in range(n_iterations + n_burn_in):
    # gibbs sampler first, and then calculate posterior
    samples = gibbs_sampler_queue(y, len(y), v, theta_dict)
    if i == 0:
      # center first draw on the expectations of the priors on thetas/etas
      eta_init = [float(min(y)), 10/2, float(np.log(1/6))]
      proposal = scipy.stats.multivariate_normal(mean = eta_init,
                                                 cov = np.diag([x ** 2 for x in eta_sd])).rvs(size = 1)
      next_state = proposal
    else:
      current_state = MC_chain[i-1]
      proposal = scipy.stats.multivariate_normal(mean = current_state,
                                                 cov = np.diag([x ** 2 for x in eta_sd])).rvs(size = 1)
      theta_current = {
          "theta_1": float(current_state[0]),
          "theta_2": float(current_state[0]) + float(current_state[1]),
          'theta_3': np.exp(float(current_state[2]))
      }
      theta_proposal = {
          'theta_1': float(proposal[0]),
          'theta_2': float(proposal[1]) + float(proposal[0]),
          'theta_3': np.exp(float(proposal[2]))
      }
      log_posterior_proposal = log_posterior(theta_proposal, samples, y)
      if log_posterior_proposal == -np.inf:
        next_state = current_state
      else:
        log_posterior_current = log_posterior(theta_current, samples, y)
        acceptance_ratio = float(min(1, np.exp(log_posterior_proposal - log_posterior_current)))
        U = float(scipy.stats.uniform().rvs(1))
        if U <= acceptance_ratio:
          next_state = proposal
        else:
          next_state = current_state
    MC_chain.append(next_state)
  # Throw out burn-in samples
  MC_chain = MC_chain[n_burn_in:]
  return MC_chain

In [112]:
eta_intermediate = [0.0764, 0.1093, 0.1441]
test_run = metropolis_simple(eta_intermediate, theta_intermediate, v_sim, Intermediate, 10, 0)

[6.04, 7.819915907695906, 4.49, 4.36, 9.86, 9.91, 5.02, 5.76, 4.67, 6.25, 4.77, 5.52, 6.1, 6.67, 6.88, 5.64, 4.42, 4.45, 4.77, 6.52, 4.76, 6.44, 4.73, 6.79, 5.05, 4.59, 4.75, 5.85, 5.42, 5.05, 6.49, 5.76, 8.67, 16.65, 4.86, 6.27, 6.26, 5.14, 10.6, 4.23, 6.15, 5.59, 6.34, 6.8, 4.39, 5.71, 5.41, 4.04, 5.01]
[False, False]
[6.04, 7.823454409230209, 4.49, 4.36, 9.86, 9.91, 5.02, 5.76, 4.67, 6.25, 4.77, 5.52, 6.1, 6.67, 6.88, 5.64, 4.42, 4.45, 4.77, 6.52, 4.76, 6.44, 4.73, 6.79, 5.05, 4.59, 4.75, 5.85, 5.42, 5.05, 6.49, 5.76, 8.67, 16.65, 4.86, 6.27, 6.26, 5.14, 10.6, 4.23, 6.15, 5.59, 6.34, 6.8, 4.39, 5.71, 5.41, 4.04, 5.01]
[True, False]
[6.04, 8.222145778435355, 4.49, 4.36, 9.86, 9.91, 5.02, 5.76, 4.67, 6.25, 4.77, 5.52, 6.1, 6.67, 6.88, 5.64, 4.42, 4.45, 4.77, 6.52, 4.76, 6.44, 4.73, 6.79, 5.05, 4.59, 4.75, 5.85, 5.42, 5.05, 6.49, 5.76, 8.67, 16.65, 4.86, 6.27, 6.26, 5.14, 10.6, 4.23, 6.15, 5.59, 6.34, 6.8, 4.39, 5.71, 5.41, 4.04, 5.01]
[False, False]
[6.04, 9.52, 4.49, 4.36, 7.36108298

In [ ]:
len(test_run)
eta_1 = [x[0] for x in test_run]
eta_2 = [y[1] for y in test_run]
eta_3 = [z[1] for z in test_run]

In [ ]:
# testing log posterior
v_sim, y_sim = simulate_v(theta_intermediate, len(Intermediate), Intermediate)
print(np.cumsum(v_sim))
print(np.cumsum(Intermediate))
logpost_intermediate = log_posterior(theta_intermediate, v_sim, Intermediate)
print(logpost_intermediate)

In [ ]:
print(np.mean(eta_2))
print(statistics.variance(eta_2))